## Testing the API

## Signing in

In [1]:
from atproto import Client

import pandas as pd
from PIL import Image
import requests
from io import BytesIO

import os

In [2]:
cwd = 'C:/Users/dorya/Documents'

In [3]:
data_folder = f'{cwd}/data'

In [4]:
client = Client()

In [5]:
with open(f'{data_folder}/usr_access') as f:
    usrname = f.read().strip()

In [6]:
with open(f'{data_folder}/key_access') as f:
    pwd = f.read().strip()

In [7]:
_ = client.login(usrname, pwd)

## Making a Post

In [8]:
# client.send_post('Hello World')

It worked!

## Building the Framework of the Bot

In [44]:
def create_metadata(row):
    if type(row.redshift.iloc[0]) == str:
        z = row.redshift.iloc[0]
    else:
        z = "%.2f" % ( row.redshift.iloc[0] )
    ra = "%.5f" % row.ra.iloc[0]
    dec = "%.5f" % row.dec.iloc[0]
    clsf = row.galaxy_description.iloc[0]
    survey = row.imaging.iloc[0]
    if "CANDELS-COODS" in survey:
        survey = 'CANDELS-GOODS'
       
    project = row.project.iloc[0]
    if 'Hubble' in project:
        instr = 'Hubble Space Telescope'
    elif 'CANDELS' in project:
        instr = 'Hubble Space Telescope'
    elif 'Galaxy Zoo 2' in project:
        instr = 'Apache Point 2.5m Telescope'

    t_lookback = row.t_lookback.iloc[0]
    if t_lookback < 1:
        tmp = t_lookback * 1000
        t_lookback_string = '%.3f million years' % tmp
    else:
        t_lookback_string = '%.2f billion years' % t_lookback
    

    metadata = (
"""A {}, observed with the {} in the {} survey.

It is at redshift {} (lookback time {}) with coordinates ({}, {}).

This classification was made in the {} project.
""").format(
            clsf, instr, survey, z, t_lookback_string, ra, dec, project
        )

    return metadata

In [45]:
## Functions 
def pull_galaxy_image(url):
    response = requests.get(url)
    img_data = BytesIO(response.content)
    return img_data 

In [46]:
def post(image, metadata, client):

    response = client.send_image(text = metadata, image = image, image_alt = 'A Galaxy')

    if len(response.errors) > 0:
        print("error posting to BlueSky -- errors:")
        print(response.errors)
    else:
        print("successfully posted animation")
    
    return response

In [47]:
# Selecting a Galaxy to upload.
cat_path = 'https://github.com/vrooje/vrooje.github.io/raw/master/files/gz2_gzh_gzc_gzbot_table.csv'
gal_row = pd.read_csv(cat_path).sample(1)
url = gal_row['image_url'].iloc[0]

In [51]:
gal_row

,subject_id,ra,dec,project,imaging,galaxy_description,redshift,t_lookback,image_url,n_class,...,p_nobar,p_spiral,p_notspiral,p_ring,p_lens,p_disturbed,p_irregular,p_other,p_merger,p_dustlane
10911,AHZ20013dn,149.511309,2.409193,GZ: Hubble,COSMOS,spiral galaxy,0.50,5.209765,http://zoo-hst.s3.amazonaws.com/20132968.jpg,40,...,0.438316,0.805304,0.0,0.0,0.0,0.027,0.0,0.027,0.027,0.027


In [49]:
# Creating the Post
image = pull_galaxy_image(url)
post_string = create_metadata(gal_row)

In [50]:
print(post_string)

A spiral galaxy, observed with the Hubble Space Telescope in the COSMOS survey.

It is at redshift 0.50 (lookback time 5.21 billion years) with coordinates (149.51131, 2.40919).

This classification was made in the GZ: Hubble project.



In [52]:
# Posting
response = post(image, post_string, client)
print(response)

AttributeError: 'CreateRecordResponse' object has no attribute 'errors'